In [1]:
import time
start_time = time.time()

In [2]:
import nltk
import scipy
from nltk.tokenize import word_tokenize
from nltk.util import ngrams
import numpy as np
import re
import pickle

Split Incoming Enteries

In [5]:
ETITLE = ["ENDTITLE"]
def split_text(text):
    entries = []
    j=0
    while (j <(len(text))):
        if (text[j].count('ENDTITLE') == 3):
            text_split = re.split("ENDTITLE|ENDTITLE|ENDTITLES|ENDABSTRACTS",text[j])
            title1 = text_split[0].strip()
            title2 = text_split[1].strip()
            title3 = text_split[2].strip()
            abstract = text_split[3].replace("\r\n"," ").strip()
            decision = text_split[4].strip()
            entries.append({u'title':title1,u'abstract':abstract,u'decision':decision})
            j = j +1
        elif (set(ETITLE).intersection(text[j].split()) == set(['ENDTITLE'])):
            text_split = re.split("ENDTITLES|ENDTITLE|ENDABSTRACTS",text[j])
            title1 = text_split[0].strip()
            title2 = text_split[1].strip()
            abstract = text_split[2].replace("\r\n"," ").strip()
            decision = text_split[3].strip()
            entries.append({u'title':title1,u'abstract':abstract,u'decision':decision})
            j = j +1
        else: 
            text_split = re.split("ENDTITLES|ENDABSTRACTS",text[j])
            title = text_split[0].strip()
            abstract = text_split[1].replace("\r\n"," ").strip()
            decision = text_split[2].strip()
            entries.append({u'title':title,u'abstract':abstract,u'decision':decision})
            j = j + 1
    return(entries)

Create n-grams

In [6]:
def BIGRAMS(string):
    token = nltk.word_tokenize(string)
    bigram = ngrams(token,2)
    bigrams = ', '.join(' '.join((a, b)) for a, b in bigram)
    biNoSpace = bigrams.replace(" ","")
    BIGRAM = biNoSpace.replace(",", " ")
    return BIGRAM
    
def TRIGRAMS(string):
    token = nltk.word_tokenize(string)
    trigram = ngrams(token, 3)
    trigrams = ', '.join(' '.join((a, b, c)) for a, b, c in trigram)
    triNoSpace = trigrams.replace(" ","")
    TRIGRAM = triNoSpace.replace(",", " ")
    return TRIGRAM

def QUADGRAMS(string):
    token = nltk.word_tokenize(string)
    quadgram = ngrams(token, 4)
    quadgrams = ', '.join(' '.join((a, b, c, d)) for a, b, c, d in quadgram)
    quadNoSpace = quadgrams.replace(" ","")
    QUADGRAM = quadNoSpace.replace(",", " ")  
    return QUADGRAM

Load any csv's

In [7]:
def load(name):
    with open(name+".csv", "rb") as f:
        File = np.loadtxt(f, dtype=str)
    return File

Function to Search:

In [8]:
ListOfWords = []
def Search(Abs, W):    
    match = set(Abs).intersection(set(W))
    Match = list(match)
    i=0
    while (i < len(Match)):
        ListOfWords.append(Match[i])
        #print(Match[i])
        i = i + 1    

thing to tie it all together:

In [9]:
def Key_Num(TEXT, DICT):
    Search(TEXT.split()             ,DICT)
    Search(BIGRAMS(TEXT).split()    ,DICT)
    Search(TRIGRAMS(TEXT).split()   ,DICT)
    Search(QUADGRAMS(TEXT).split()  ,DICT)

    score = (len(ListOfWords))
    #print("The amount of terms found is {0}" .format(score))
    return score

function to import data:

In [10]:
Data = []
with open("small.txt", "r") as ins:
    for line in ins:
        line = unicode(line, errors='replace')
        Data.append(line)

Test with abstract

In [11]:
#text1 = "Cellular non-nonlinear network model of microbial fuel cellENDTITLECellular non-nonlinear network model of microbial fuel cell ENDTITLES A cellular non-linear network (CNN) is a uniform regular array of locally\r\nconnected continuous-state machines, or nodes, which update their states\r\nsimultaneously in discrete time. A microbial fuel cell (MFC) is an\r\nelectro-chemical reactor using the metabolism of bacteria to drive an\r\nelectrical current. In a CNN model of the MFC, each node takes a vector of\r\nstates which represent geometrical characteristics of the cell, like the\r\nelectrodes or impermeable borders, and quantify measurable properties like\r\nbacterial population, charges produced and hydrogen ions concentrations. The\r\nmodel allows the study of integral reaction of the MFC, including temporal\r\noutputs, to spatial disturbances of the bacterial population and supply of\r\nnutrients. The model can also be used to evaluate inhomogeneous configurations\r\nof bacterial populations attached on the electrode biofilms. ENDABSTRACTS Rejected"
#text2 = "non-nonlinear network model of microbial fuel cellENDTITLECellular non-nonlinear network model of microbial fuel cell ENDTITLES A cellular non-linear network (CNN) is a uniform regular array of locally\r\nconnected continuous-state machines, or nodes, which update their states\r\nsimultaneously in discrete time. A microbial fuel cell (MFC) is an\r\nelectro-chemical reactor using the metabolism of bacteria to drive an\r\nelectrical current. In a CNN model of the MFC, each node takes a vector of\r\nstates which represent geometrical characteristics of the cell, like the\r\nelectrodes or impermeable borders, and quantify measurable properties like\r\nbacterial population, charges produced and hydrogen ions concentrations. The\r\nmodel allows the study of integral reaction of the MFC, including temporal\r\noutputs, to spatial disturbances of the bacterial population and supply of\r\nnutrients. The model can also be used to evaluate inhomogeneous configurations\r\nof bacterial populations attached on the electrode biofilms. ENDABSTRACTS Rejected"
#text  = [text1,text2]

TEXT = split_text(Data)

TITLE = []
ABSTRACT = []
DECISION = []

for r in range(len(Data)):
    TITLE.append(TEXT[r].get('title').lower())
    ABSTRACT.append(TEXT[r].get('abstract').lower())
    DECISION.append(TEXT[r].get('decision').lower())

key = load("KeyWords")
q=0
while (q < len(key)):
    key[q] = key[q].lower()
    q = q + 1

In [12]:
RESULTS = [] 
for i in range(len(Data)):
    RESULTS.append(Key_Num(TITLE[i], key))
    ListOfWords = []
    RESULTS.append(Key_Num(ABSTRACT[i], key))
    ListOfWords=[]
    RESULTS.append(DECISION[i])

for n, i in enumerate(RESULTS):
    if i == "rejected" :
        RESULTS[n] = 0
    if i == "core" :
        RESULTS[n] = 2
    if i == "non-core" :
        RESULTS[n] = 1

RESULT_array=np.reshape(RESULTS, (len(Data),3))

not every entry tell us if it is core, so it sucks and we can get rid of it:

In [13]:
RESULTS_ARRAY = []
m = 0
while (m < len(RESULT_array)):
    if (RESULT_array[m][2] != ''):
        RESULTS_ARRAY.append(RESULT_array[m])
    m = m + 1

function to save everything:

In [14]:
def output_file(name,output):
    with open(name+".pkl",'wb') as f:
        pickle.dump(output,f)
        
output_file('keyword_occurances',RESULTS_ARRAY)

In [15]:
with open('keyword_occurances.pkl','rb') as f:
    mydata= pickle.load(f)

print mydata
print len(mydata)

[array([u'2', u'7', u'0'],
      dtype='<U21'), array([u'0', u'5', u'2'],
      dtype='<U21'), array([u'0', u'0', u'0'],
      dtype='<U21'), array([u'2', u'8', u'0'],
      dtype='<U21'), array([u'2', u'10', u'2'],
      dtype='<U21'), array([u'3', u'0', u'2'],
      dtype='<U21'), array([u'1', u'0', u'2'],
      dtype='<U21'), array([u'1', u'4', u'1'],
      dtype='<U21'), array([u'0', u'0', u'1'],
      dtype='<U21'), array([u'0', u'0', u'2'],
      dtype='<U21'), array([u'0', u'0', u'2'],
      dtype='<U21'), array([u'0', u'0', u'2'],
      dtype='<U21'), array([u'0', u'21', u'0'],
      dtype='<U21'), array([u'1', u'5', u'0'],
      dtype='<U21'), array([u'3', u'0', u'2'],
      dtype='<U21'), array([u'1', u'23', u'2'],
      dtype='<U21'), array([u'2', u'9', u'2'],
      dtype='<U21'), array([u'1', u'21', u'2'],
      dtype='<U21'), array([u'5', u'11', u'2'],
      dtype='<U21'), array([u'2', u'14', u'2'],
      dtype='<U21'), array([u'2', u'6', u'2'],
      dtype='<U21'), array(

In [16]:
print("My program took", time.time() - start_time, "to run")

('My program took', 122.73772501945496, 'to run')
